# Modelando precios de casas - Navent Dataset

## Introducción

Vamos a trabajar en equipo para obtener el mejor resultado posible en la competencia de Valuación de casas de Fundacion Sadosky: https://metadata.fundacionsadosky.org.ar/competition/4/.

El objetivo será generar un modelo de regresión que nos acerque lo más posible al top ten de la competencia con la métrica seleccionada (RMSLE).

Antes de empezar vamos a tener que tener en cuenta algunos puntos:

- La competencia ya finalizó, por lo que no podemos cargar nuestros datos en la página y comparar directamente con los resultados del dataset de test. Por ello, dividiremos el dataset de training en dos: el que usaremos para modelar, y uno hold-out para validar que usaremos para comparar los mejores modelos que ustedes hayan generado.
- Vamos a trabajar en equipo, con la idea de que esto ayude a la comprensión del problema y a la búsquedas de estrategias para mejorar nuestro modelo, es sumamente importante que todos participen y discutan las decisiones que toman.
- Vamos a trabajar con este dataset en las próximas 3 clases, incorporando nuevas técnicas y conceptos para intentar mejorar nuestros modelos, por lo que no tienen que apurarse para resolver todo en la primera semana, tienen tiempo de analizar el problema e ir iterando en las posibles soluciones.
- Por último, recuerden que es un proceso iterativo, es recomendable que empiecen simple, lleguen a entrenar un modelo no tan complejo, obtengan un resultado, y luego vayan avanzando sobre eso.

## Actividades

1. Repasar el notebook introductorio provisto por Navent:
https://metadata.fundacionsadosky.org.ar/media/navent/metaDataNavent.ipynb

2. Investigar acerca de la métrica seleccionada (RMSLE): https://hrngok.github.io/posts/metrics/

3. Dividir el dataset en dos partes: una parte para training-validation, y una parte para hold-out, ésta última debe tener el 25% de los datos. Deben usar train_test_split de Scikit Learn con random_state=42 (esto es sólo para que todos tengamos el mismo split y las comparaciones entre sus modelos sean equivalentes)

4. Aplicar pre procesamiento que consideren necesario sobre los datos. Pueden usar como ejemplo lo que está hecho en el notebook del punto 1, pero sería interesante que tomen decisiones basados en un análisis propio: ¿agregarían o quitarían features? ¿Cambiarían algo del procesamiento de las features categóricas?

5. Modelar el problema. Deben entrenar modelos usando cross validation y grid/random search, para ello deben seleccionar que algoritmos de regresión van a probar: hasta ahora sólo hemos visto Regresión Lineal y Árboles de decisión, pero pueden seleccionar otro modelo lineal de la librería Scikit Learn: https://scikit-learn.org/stable/modules/classes.html#module-sklearn.linear_model. Acá sería bueno que investiguen respecto a algoritmos que vayan a elegir: ¿Que aportan sobre una regresión lineal tradicional? ¿Qué hiperparámetros se pueden configurar? 

6. Luego de entrenar con validación cruzada y búsqueda de hiperparámetros, comprobar el funcionamiento del modelo sobre el dataset hold out. Recuerden que esto debe hacerse solo al final para comparar modelos, no para optimizar hiperparámetros, por lo que es el paso final del proceso.

7. Generar un notebook con explicaciones claras del proceso realizado, las decisiones que se tomaron, y el resultado obtenido. 


In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
data = pd.read_csv('./Data/train.csv')
data.head()

,id,titulo,descripcion,tipodepropiedad,direccion,ciudad,provincia,antiguedad,habitaciones,garages,...,idzona,lat,lng,fecha,gimnasio,usosmultiples,piscina,escuelascercanas,centroscomercialescercanos,precio
0,254099,depto. tipo a-402,"depto. interior de 80.15m2, consta de sala com...",Apartamento,Avenida Division del Norte 2005,Benito Juárez,Distrito Federal,NaN,2.0,1.0,...,23533.0,NaN,NaN,2015-08-23 00:00:00,0.0,0.0,0.0,0.0,0.0,2273000.0
1,53461,condominio horizontal en venta,"<p>entre sonora y guerrero, atr&aacute;s del h...",Casa en condominio,AV. MEXICO,La Magdalena Contreras,Distrito Federal,10.0,3.0,2.0,...,24514.0,19.310205,-99.227655,2013-06-28 00:00:00,0.0,0.0,0.0,1.0,1.0,3600000.0
2,247984,casa en venta urbi 3 recamaras tonala,descripcion \nla mejor ubicacion residencial e...,Casa,Urbi Tonala,Tonalá,Jalisco,5.0,3.0,2.0,...,48551.0,NaN,NaN,2015-10-17 00:00:00,0.0,0.0,0.0,0.0,0.0,1200000.0
3,209067,casa sola en toluca zinacantepec con credito i...,casa en privada con caseta de vigilancia casas...,Casa,IGNACIO MANUEL ALTAMIRANO 128,Zinacantepec,Edo. de México,1.0,2.0,1.0,...,53666.0,19.301890,-99.688015,2012-03-09 00:00:00,0.0,0.0,0.0,1.0,1.0,650000.0
4,185997,paseos del sol,bonito departamento en excelentes condiciones ...,Apartamento,PASEOS DEL SOL,Zapopan,Jalisco,10.0,2.0,1.0,...,47835.0,NaN,NaN,2016-06-07 00:00:00,0.0,0.0,0.0,0.0,0.0,1150000.0


In [3]:
from sklearn.model_selection import train_test_split
data_train, data_holdout = train_test_split(data, test_size=0.25, random_state=42)

In [4]:
data_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 180000 entries, 122812 to 121958
Data columns (total 23 columns):
id                            180000 non-null int64
titulo                        175957 non-null object
descripcion                   178759 non-null object
tipodepropiedad               179968 non-null object
direccion                     140232 non-null object
ciudad                        179732 non-null object
provincia                     179888 non-null object
antiguedad                    147355 non-null float64
habitaciones                  163247 non-null float64
garages                       151461 non-null float64
banos                         160379 non-null float64
metroscubiertos               166943 non-null float64
metrostotales                 141329 non-null float64
idzona                        158521 non-null float64
lat                           87510 non-null float64
lng                           87510 non-null float64
fecha                         

In [5]:
nulls = pd.DataFrame(data_train.isnull().sum().sort_values(), columns=['nulls'])
nulls['porcentaje'] = round(100*nulls['nulls'] / len(data_train), 2)
nulls

,nulls,porcentaje
id,0,0.00
escuelascercanas,0,0.00
piscina,0,0.00
usosmultiples,0,0.00
gimnasio,0,0.00
fecha,0,0.00
centroscomercialescercanos,0,0.00
precio,0,0.00
tipodepropiedad,32,0.02
provincia,112,0.06


In [6]:
X = data_train.drop(['fecha','id','lat','lng','direccion', 'descripcion', 'titulo','idzona','precio'], axis=1)
y = data_train['precio']

In [7]:
X.head()

,tipodepropiedad,ciudad,provincia,antiguedad,habitaciones,garages,banos,metroscubiertos,metrostotales,gimnasio,usosmultiples,piscina,escuelascercanas,centroscomercialescercanos
122812,Casa en condominio,Cancún,Quintana Roo,NaN,3.0,2.0,2.0,170.0,160.0,0.0,0.0,0.0,0.0,0.0
180398,Casa,Guaymas,Sonora,5.0,2.0,1.0,2.0,140.0,156.0,0.0,0.0,0.0,1.0,1.0
157317,Apartamento,Benito Juárez,Distrito Federal,20.0,3.0,1.0,2.0,93.0,NaN,0.0,0.0,0.0,1.0,1.0
78425,Casa,Cuautitlán Izcalli,Edo. de México,5.0,3.0,1.0,1.0,113.0,85.0,0.0,0.0,0.0,0.0,0.0
73755,Casa,Coyoacán,Distrito Federal,5.0,4.0,3.0,2.0,150.0,400.0,0.0,0.0,0.0,1.0,1.0


In [8]:
X["metroscubiertos"], X["metrostotales"] = np.where(X['metroscubiertos'] > X['metrostotales'], [X["metrostotales"], X["metroscubiertos"]], [X["metroscubiertos"], X["metrostotales"] ])
X.head()

,tipodepropiedad,ciudad,provincia,antiguedad,habitaciones,garages,banos,metroscubiertos,metrostotales,gimnasio,usosmultiples,piscina,escuelascercanas,centroscomercialescercanos
122812,Casa en condominio,Cancún,Quintana Roo,NaN,3.0,2.0,2.0,160.0,170.0,0.0,0.0,0.0,0.0,0.0
180398,Casa,Guaymas,Sonora,5.0,2.0,1.0,2.0,140.0,156.0,0.0,0.0,0.0,1.0,1.0
157317,Apartamento,Benito Juárez,Distrito Federal,20.0,3.0,1.0,2.0,93.0,NaN,0.0,0.0,0.0,1.0,1.0
78425,Casa,Cuautitlán Izcalli,Edo. de México,5.0,3.0,1.0,1.0,85.0,113.0,0.0,0.0,0.0,0.0,0.0
73755,Casa,Coyoacán,Distrito Federal,5.0,4.0,3.0,2.0,150.0,400.0,0.0,0.0,0.0,1.0,1.0


In [9]:
X.tipodepropiedad.value_counts()

Casa                             106225
Apartamento                       43184
Casa en condominio                14490
Terreno                            7411
Local Comercial                    2254
Oficina comercial                  1307
Edificio                           1072
Bodega comercial                   1049
Terreno comercial                   961
Casa uso de suelo                   519
Quinta Vacacional                   285
Villa                               253
Duplex                              247
Inmuebles productivos urbanos       145
Local en centro comercial           128
Rancho                              123
Departamento Compartido             107
Otros                               104
Nave industrial                      57
Terreno industrial                   26
Huerta                               15
Lote                                  5
Hospedaje                             1
Name: tipodepropiedad, dtype: int64

In [10]:
X.tipodepropiedad.isna().sum()

32

In [11]:
X.tipodepropiedad.dropna(inplace=True)

In [12]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
imp = IterativeImputer(missing_values=np.nan, max_iter=20)
X[['antiguedad', 'habitaciones','garages','banos','metroscubiertos','metrostotales','gimnasio','usosmultiples','piscina','escuelascercanas','centroscomercialescercanos']] = imp.fit_transform(X[['antiguedad', 'habitaciones','garages','banos','metroscubiertos','metrostotales','gimnasio','usosmultiples','piscina','escuelascercanas','centroscomercialescercanos']])
X.head()

,tipodepropiedad,ciudad,provincia,antiguedad,habitaciones,garages,banos,metroscubiertos,metrostotales,gimnasio,usosmultiples,piscina,escuelascercanas,centroscomercialescercanos
122812,Casa en condominio,Cancún,Quintana Roo,7.823792,3.0,2.0,2.0,160.0,170.00000,0.0,0.0,0.0,0.0,0.0
180398,Casa,Guaymas,Sonora,5.000000,2.0,1.0,2.0,140.0,156.00000,0.0,0.0,0.0,1.0,1.0
157317,Apartamento,Benito Juárez,Distrito Federal,20.000000,3.0,1.0,2.0,93.0,140.00816,0.0,0.0,0.0,1.0,1.0
78425,Casa,Cuautitlán Izcalli,Edo. de México,5.000000,3.0,1.0,1.0,85.0,113.00000,0.0,0.0,0.0,0.0,0.0
73755,Casa,Coyoacán,Distrito Federal,5.000000,4.0,3.0,2.0,150.0,400.00000,0.0,0.0,0.0,1.0,1.0


In [13]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
X['tipodepropiedad'] = le.fit_transform((X['tipodepropiedad'].astype(str)))
X['ciudad'] = le.fit_transform((X['ciudad'].astype(str)))
X['provincia'] = le.fit_transform((X['provincia'].astype(str)))
X

,tipodepropiedad,ciudad,provincia,antiguedad,habitaciones,garages,banos,metroscubiertos,metrostotales,gimnasio,usosmultiples,piscina,escuelascercanas,centroscomercialescercanos
122812,3,109,22,7.823792,3.0,2.0,2.0,160.0,170.00000,0.0,0.0,0.0,0.0,0.0
180398,2,259,25,5.000000,2.0,1.0,2.0,140.0,156.00000,0.0,0.0,0.0,1.0,1.0
157317,0,87,8,20.000000,3.0,1.0,2.0,93.0,140.00816,0.0,0.0,0.0,1.0,1.0
78425,2,196,10,5.000000,3.0,1.0,1.0,85.0,113.00000,0.0,0.0,0.0,0.0,0.0
73755,2,188,8,5.000000,4.0,3.0,2.0,150.0,400.00000,0.0,0.0,0.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119879,2,109,22,5.000000,3.0,2.0,3.0,182.0,225.00000,0.0,0.0,1.0,1.0,1.0
103694,2,636,10,0.000000,2.0,1.0,1.0,59.0,59.00000,0.0,0.0,0.0,0.0,1.0
131932,2,476,14,1.000000,3.0,2.0,2.0,105.0,105.00000,0.0,0.0,1.0,1.0,1.0
146867,2,636,10,10.000000,4.0,1.0,2.0,80.0,80.00000,0.0,0.0,0.0,1.0,1.0


In [14]:
from sklearn.impute import IterativeImputer
#imp = IterativeImputer(missing_values=np.nan,max_iter=15)
#transformed = imp.fit_transform(X[[]])
#b_df[[]] = transformed
#b_df